## LCEL 

> https://www.youtube.com/watch?v=8aUYzb1aYDU

---

In [45]:
import os
import time
from operator import itemgetter

from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnableLambda, RunnablePassthrough
from langchain.document_loaders import TextLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma.vectorstores import Chroma
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

In [4]:
GROQ_API_KEY = ""
os.environ['GROQ_API_KEY'] = GROQ_API_KEY

In [5]:
llm = ChatGroq(
    model="mixtral-8x7b-32768",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [6]:
messages = [
    (
        "system",
        "You are a helpful assistant that translates English to French. Translate the user sentence.",
    ),
    ("human", "I love programming."),
]
ai_msg = llm.invoke(messages)
ai_msg

AIMessage(content='I love programming in French is: "J\'aime programmer."', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 30, 'total_tokens': 46, 'completion_time': 0.024077454, 'prompt_time': 0.002665191, 'queue_time': 0.012173658, 'total_time': 0.026742645}, 'model_name': 'mixtral-8x7b-32768', 'system_fingerprint': 'fp_c5f20b5bb1', 'finish_reason': 'stop', 'logprobs': None}, id='run-d8c20fea-cf5a-4015-9bb1-944c297ac975-0', usage_metadata={'input_tokens': 30, 'output_tokens': 16, 'total_tokens': 46})

### Simple chains using LCEL

In [10]:
template = "Hi! I am learning {skill}. Can you help point me to top 5 courses for it."
prompt = PromptTemplate(template=template, input_variables=['skill'])
print (prompt)

input_variables=['skill'] input_types={} partial_variables={} template='Hi! I am learning {skill}. Can you help point me to top 5 courses for it.'


In [11]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

/var/folders/9g/tslm33j92p1blxwl8vvjxn1r0000gn/T/ipykernel_18185/4035131671.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(prompt=prompt, llm=llm)


In [14]:
print (llm_chain.run({"skill": "Video editing"}))

Hello! I'd be happy to help you find some top-notch video editing courses. Here are five courses that I highly recommend:

1. Adobe Premiere Pro Masterclass: Learn Video Editing in Premiere Pro - This course is great for beginners who want to learn Adobe Premiere Pro, one of the most popular video editing software. It covers all the basics and advanced techniques you need to know to create professional-looking videos.
2. Video Editing with Adobe Premiere Pro for Beginners: Start Editing Video Like a Pro - This course is another excellent option for beginners who want to learn Adobe Premiere Pro. It covers the fundamentals of video editing, including cutting, transitions, and color correction.
3. Final Cut Pro X: Video Editing Masterclass - If you're a Mac user, this course is an excellent choice for learning Final Cut Pro X, another popular video editing software. It covers everything from basic editing techniques to advanced color grading and audio mixing.
4. Video Editing in DaVinci 

In [19]:
chain = prompt | llm | StrOutputParser()

print (chain.invoke({"skill": "ML Engineering"}))

Hello! I'm glad to hear that you're interested in machine learning engineering. Here are five top courses that can help you get started:

1. Machine Learning Engineering on Coursera: This course is offered by the University of Washington and covers the entire machine learning engineering pipeline, from data preparation to model deployment. It includes hands-on projects and is a great way to get started with machine learning engineering.
2. Machine Learning on edX: This course is offered by MIT and covers the fundamentals of machine learning, including supervised and unsupervised learning, linear models, and neural networks. It's a great course for those who want to build a strong foundation in machine learning before diving into engineering.
3. Deep Learning Specialization on Coursera: This is a five-course specialization offered by Andrew Ng, a pioneer in the field of deep learning. The courses cover neural networks, deep learning, structuring machine learning projects, convolutional 

In [24]:
# each elem in the LCEL should have an `invoke()` method

"invoke" in dir(prompt), "invoke" in dir(llm), "invoke" in dir(StrOutputParser())

(True, True, True)

### Runnables

In [21]:
chain = RunnablePassthrough()

In [25]:
"invoke" in dir(RunnablePassthrough())

True

In [26]:
chain.invoke("hi")

'hi'

In [27]:
chain = RunnablePassthrough() | RunnablePassthrough() | RunnablePassthrough()

In [28]:
chain.invoke("mini")

'mini'

`RunnablePassthrough` is used to take an input and pass that to the next stage

In [29]:
def string_upper(input_str: str) -> str:
    return input_str.upper()

`RunnableLambda` is used to run functions

In [30]:
chain = RunnablePassthrough() | RunnableLambda(string_upper)

In [31]:
chain.invoke("mini")

'MINI'

In [32]:
"invoke" in dir(string_upper)

False

In [33]:
chain = RunnableParallel({"x": RunnablePassthrough(), "y": RunnablePassthrough()})
chain.invoke("mini")

{'x': 'mini', 'y': 'mini'}

In [34]:
chain = RunnableParallel({"details": RunnablePassthrough(), "age": lambda x: x['age']})
chain.invoke({"name": "mini", "age": 28})

{'details': {'name': 'mini', 'age': 28}, 'age': 28}

In [38]:
def fetch_name(x):
    return x.get("food", "not found")

In [41]:
chain = RunnableParallel({
    "food": RunnablePassthrough() | RunnableLambda(fetch_name),
    "age": lambda x: x['age']
})

In [42]:
chain.invoke({"name": "mini", "age": 28})

{'food': 'not found', 'age': 28}

### Setup Basic RAG infra

In [11]:
loader = DirectoryLoader(
    path = "./data", glob="./*.txt", loader_cls=TextLoader
)

docs = loader.load()

In [12]:
len(docs)

1

In [14]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=10,
    length_function=len
)

new_docs = text_splitter.split_documents(
    documents=docs
)

print (len(new_docs))

103


In [17]:
model_name = "BAAI/bge-base-en-v1.5"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/Users/rishabhagarwal/.pyenv/versions/3.12.4/envs/poetry-env/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [18]:
db = Chroma.from_documents(
    documents=new_docs, embedding=embeddings
)

In [19]:
db._persist_directory

'./chroma'

In [20]:
type(db)

langchain_chroma.vectorstores.Chroma

In [24]:
retriever = db.as_retriever(
    search_kwargs = {
        "k": 4,
        
    }
)

In [27]:
template = """
Answer the following question based ONLY on the retrieved context.
If the answer does not occur in the context, reply: I do not know

Context:
{context}

Question:
{question}

Answer in proper markdown format:
"""

prompt = PromptTemplate.from_template(template)

In [28]:
retriever.invoke("hi")

[Document(metadata={'source': 'data/essay.txt'}, page_content="When I got back to New York I resumed my old life, except now I was rich. It was as weird as it sounds. I resumed all my old patterns, except now there were doors where there hadn't been. Now when I was tired of walking, all I had to do was raise my hand, and (unless it was raining) a taxi would stop to pick me up. Now when I walked past charming little restaurants I could go in and order lunch. It was exciting for a while. Painting started to go better. I experimented with a new kind of still life where I'd paint one painting in the old way, then photograph it and print it, blown up, on canvas, and then use that as the underpainting for a second still life, painted from the same objects (which hopefully hadn't rotted yet)."),
 Document(metadata={'source': 'data/essay.txt'}, page_content="I was nervous about money, because I could sense that Interleaf was on the way down. Freelance Lisp hacking work was very rare, and I did

### RAG LCEL

In [29]:
retrieval_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | prompt
    | llm
    | StrOutputParser()
)

In [37]:
s_time = time.perf_counter()

question = "What was the author's takeaway from YCombinator?"
print(retrieval_chain.invoke(question))
e_time = time.perf_counter()
elapsed_time = e_time - s_time
print(f"Elapsed time: {elapsed_time:.4f} seconds")

The author's takeaway from YCombinator was that it was an engaging and effective way to learn about startups in a short amount of time due to the varied problems presented by the new batches of startups every 6 months. They also realized that YCombinator would eventually take up all of their attention, leading them to the decision that it couldn't be their life's work and they would have to leave eventually. Additionally, the author mentioned that YCombinator was renamed from Cambridge Seed to avoid a regional name and to differentiate from the staid color choices of other VCs. YCombinator also became a fund for a couple of years starting in 2009 due to its growing size, but later returned to being self-funded.
Elapsed time: 0.8561 seconds


In [38]:
s_time = time.perf_counter()

question = "What was the author's takeaway from YCombinator?"
print(await retrieval_chain.ainvoke(question))
e_time = time.perf_counter()
elapsed_time = e_time - s_time
print(f"Elapsed time: {elapsed_time:.4f} seconds")

The author's takeaway from YCombinator was that it was an engaging and effective way to learn about startups in a short amount of time due to the varied problems presented by the new batches of startups every 6 months. They also realized that YCombinator would eventually take up all of their attention, leading them to the decision that it couldn't be their life's work and they would have to leave eventually. Additionally, the author mentioned that YCombinator was renamed from Cambridge Seed to avoid a regional name and to differentiate from the staid color choices of other VCs. YCombinator also became a fund for a couple of years starting in 2009 due to its growing size, but later returned to being self-funded.
Elapsed time: 0.8729 seconds


### Using `Itemgetter`

In [40]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""
prompt = PromptTemplate.from_template(template)

In [41]:
retrieval_chain = (
    RunnableParallel({"context": itemgetter("question") | retriever,
                      "question": itemgetter("question"),
                      "language": itemgetter("language")
                      })
    | prompt
    | llm
    | StrOutputParser()
)

In [46]:
retrieval_chain.get_graph().print_ascii()

                  +------------------------------------------+            
                  | Parallel<context,question,language>Input |            
                  +------------------------------------------+            
                        *****           *           *****                 
                    ****               *                 *****            
                 ***                   *                      ****        
       +--------+                      *                          ***     
       | Lambda |                      *                            *     
       +--------+                      *                            *     
            *                          *                            *     
            *                          *                            *     
            *                          *                            *     
+----------------------+          +--------+                   +--------+ 
| VectorStoreRetriever | 

In [48]:

### itemgetter only works with dictionaries , input has to be a dict

response = retrieval_chain.invoke({'question': "What was the author's takeaway from YCombinator?",
                        'language': "bengali"})

print(response)

YCombinator-er autor ke nischoyeke bouddhho holo, YCombinator ekjon bhalo probaashho holo, kintu tini je shohoj holo na, tini ekdin birstho korte hobe. YCombinator tinike shohoj holo karon tini shohoj korechilo na, bhalo probaashho holo karon tini bhalo shikhte pari ekadin startups-er shohoj probaashho.

(Note: This is a rough translation of the answer in Bengali language. The actual translation may vary based on the dialect and context.)


In [49]:

template = 'Hi! I am learning {skill}. Can you suggest me top 5 things to learn?\n'

prompt = PromptTemplate.from_template(template=template)

chain = prompt | llm

In [54]:
for s in chain.stream({'skill':'Big Data'}):
    print(s.content,end='')

Hello! That's great to hear that you're learning Big Data. Here are the top 5 things that I would recommend you to learn:

1. **Hadoop Ecosystem**: Hadoop is a popular open-source framework for storing and processing large datasets. You should learn the core components of the Hadoop ecosystem, such as HDFS (Hadoop Distributed File System), MapReduce, YARN (Yet Another Resource Negotiator), and Hive.
2. **Spark**: Spark is a fast and general-purpose cluster computing system that can be used for a wide range of big data processing tasks, such as batch processing, interactive queries, streaming, machine learning, and graph processing. Spark provides an API for Java, Scala, Python, and R.
3. **Data Processing and Analysis Tools**: You should learn data processing and analysis tools such as Pig, Impala, and Apache Drill. These tools provide high-level abstractions for data processing and analysis, making it easier for developers and analysts to work with large datasets.
4. **Data Streaming*

In [57]:
llm2 = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.3,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [58]:
llm2.invoke("hello")

AIMessage(content="Hello! It's nice to meet you. Is there something I can help you with, or would you like to chat?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 11, 'total_tokens': 37, 'completion_time': 0.021666667, 'prompt_time': 0.001505024, 'queue_time': 0.014133766999999998, 'total_time': 0.023171691}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'stop', 'logprobs': None}, id='run-fc90c90b-72a4-4207-bd02-9e5ebced9b98-0', usage_metadata={'input_tokens': 11, 'output_tokens': 26, 'total_tokens': 37})

In [61]:
llm_judge = ChatGroq(
    model="llama3-8b-8192",
    temperature=0.0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

## Mental framework to building chains with LangChain Expression Language (LCEL), with branching and merging chains as examples

> https://medium.com/@james.li/mental-model-to-building-chains-with-langchain-expression-language-lcel-with-branching-and-36f185134eac

### Example 1 — Split into 2 branches and combine


Given a question, ask different LLM models for answer, then combine the viewpoints to form a final answer.

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*OiX1zwXXo8XGOjLYh57P8Q.png)

In [84]:
prompt = PromptTemplate.from_template("{question}")
combine_answers_template = """

Given the question and the answer from 2 different LLM systems (llm_1 and llm_2), pick the answer which better addresses the user query.
Things to judge:
1. The core answer - how relevant it is to user query
2. Tone and format
3. Explanations and other insights provided

Output the judgement in a valid JSON as shown below

```
{{
    "winner": "winner llm (llm_1 or llm_2)"
    "reason": "reason why the winner answer is better"
}}
```

Question:
{question}

Answer llm_1:
{answer_1}

Answer_llm_2:
{answer_2}

Output:

"""

judge_answers_prompt = PromptTemplate(template=combine_answers_template, input_variables=['answer_1', 'answer_2', 'question'])

In [85]:
judge_answers_prompt.invoke({
    "question": "q",
    "answer_1": "answer_1",
    "answer_2": "answer_2"
})

StringPromptValue(text='\n\nGiven the question and the answer from 2 different LLM systems (llm_1 and llm_2), pick the answer which better addresses the user query.\nThings to judge:\n1. The core answer - how relevant it is to user query\n2. Tone and format\n3. Explanations and other insights provided\n\nOutput the judgement in a valid JSON as shown below\n\n```\n{\n    "winner": "winner llm (llm_1 or llm_2)"\n    "reason": "reason why the winner answer is better"\n}\n```\n\nQuestion:\nq\n\nAnswer llm_1:\nanswer_1\n\nAnswer_llm_2:\nanswer_2\n\nOutput:\n\n')

In [86]:
model1 = llm
model2 = llm2
model3 = llm_judge

In [87]:
chain1 = prompt | model1 | StrOutputParser()
chain2 = prompt | model2 | StrOutputParser()
chain_judge = judge_answers_prompt | model3 | StrOutputParser()

#### Test each subchain separately

In [88]:
question = "What's the best way to stay up to date with latest Large Language Model news? Please keep the answer short and concise, limit to 3 bullet points."

answer_1 = chain1.invoke(question)
answer_2 = chain2.invoke(question)

In [89]:
print (answer_1)

1. Follow reputable AI research organizations and leading researchers in the field on social media, such as Twitter.
2. Regularly check AI-focused news outlets and blogs for updates on large language models.
3. Consider subscribing to relevant newsletters or email updates from organizations such as Hugging Face, OpenAI, and Google Research.


In [90]:
print (answer_2)

Here are three ways to stay up to date with the latest Large Language Model (LLM) news:

• **Follow industry leaders and researchers on Twitter**: Many prominent researchers and industry leaders in the field of LLMs share their latest findings, updates, and insights on Twitter. Follow accounts like @huggingface, @nytimes, @MIT_CSAIL, and @StanfordNLP.

• **Subscribe to AI and NLP newsletters**: Newsletters like The AI Alignment Newsletter, The NLP Newsletter, and AI in Industry provide regular updates on the latest developments in LLMs and related fields.

• **Attend online conferences and webinars**: Attend online conferences and webinars focused on LLMs and NLP to stay up to date with the latest research and advancements. Some popular events include the annual Conference on Neural Information Processing Systems (NIPS) and the International Conference on Computational Linguistics (COLING).


In [91]:
eval_output = chain_judge.invoke({
    "question": question,
    "answer_1": answer_1,
    "answer_2": answer_2
}) 

In [92]:
eval(eval_output)

{'winner': 'llm_2',
 'reason': 'The answer from llm_2 provides more specific and actionable suggestions, including the names of industry leaders and researchers to follow on Twitter, and specific newsletters and conferences to attend. The answer from llm_1 is more general and lacks the same level of detail and specificity.'}

#### Combine chains

In [96]:
combined_chain = {
    "question": RunnablePassthrough(),
    "answer_1": chain1,
    "answer_2": chain2
} | chain_judge 

In [97]:
combined_answer = combined_chain.invoke(question)
print (combined_answer)

Here is the output:

{
"winner": "llm_2",
"reason": "The answer from llm_2 provides more specific and actionable suggestions, including the names of prominent researchers and newsletters. The format is also more engaging, with bullet points and a clear structure. Additionally, llm_2's answer provides more insights and resources, such as online forums and discussion groups, which can be valuable for users looking to stay up-to-date with the latest LLM news."
}


> Tips: You can use `RunnablePassthrough.assign(input_variable: subchain)` to add additional variables as you move down the chain

You can think of `RunnablePassthrough.assign(foo=bar_chain)` as an additional dictionary item `{ "foo": "output_from_bar_chain" }`.

![](https://miro.medium.com/v2/resize:fit:1400/format:webp/1*nrCEcZU_FXlJzvygP4XFYw.png)


In [104]:
combined_chain = (
    {"question": RunnablePassthrough()}
    | RunnablePassthrough.assign(answer_1=chain1)
    | RunnablePassthrough.assign(answer_2=chain2)
    | chain_judge
)

# This is equivalent to 
# {
#    "question": RunnablePassthrough(),
#    "answer_1": chain1,
#    "answer_2": chain2,
# } | chain_judge

In [105]:
combined_answer = combined_chain.invoke(question)
print (combined_answer)

{
"winner": "llm_2",
"reason": "The answer from llm_2 provides more specific and actionable information, including specific Twitter handles and newsletters, which makes it easier for the user to stay up-to-date with the latest LLM news. Additionally, the answer from llm_2 provides more context and explanations, such as the importance of attending conferences and webinars, which makes it a more comprehensive and informative answer."
}


### Example 2 — Split into n branches and merge


In this example, given a user question, the goal is for the LLM to come up with an answer which considers both pros and cons to the question.

- The LLM first outputs a list of bullet points for a given question.
- For each bullet point, we ask the LLM to generate some pros and cons in parallel.
- Finally, we take into account the arguments on both sides to reach a final answer
